In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10

In [23]:
from peft import LoraConfig, get_peft_model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
device = torch.device("cuda:0")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir /content/GPTcheck
!tar -xvzf /content/drive/MyDrive/Colab_Notebooks/Хакатон_Санкт-Петербург/checkpoint-75epoch_small.tar.gz -C /content/GPTcheck

content/checkpoint/checkpoint-20000/
content/checkpoint/checkpoint-20000/vocab.json
content/checkpoint/checkpoint-20000/generation_config.json
content/checkpoint/checkpoint-20000/trainer_state.json
content/checkpoint/checkpoint-20000/scheduler.pt
content/checkpoint/checkpoint-20000/training_args.bin
content/checkpoint/checkpoint-20000/special_tokens_map.json
content/checkpoint/checkpoint-20000/pytorch_model.bin
content/checkpoint/checkpoint-20000/rng_state.pth
content/checkpoint/checkpoint-20000/added_tokens.json
content/checkpoint/checkpoint-20000/optimizer.pt
content/checkpoint/checkpoint-20000/tokenizer_config.json
content/checkpoint/checkpoint-20000/merges.txt
content/checkpoint/checkpoint-20000/config.json
content/checkpoint/checkpoint-20000/tokenizer.json


In [5]:
#GPT
model_gpt = AutoModelForCausalLM.from_pretrained("/content/GPTcheck/content/checkpoint/checkpoint-20000")
tokenizer_gpt = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
#LLaMa2
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model_llama = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model_llama.config.use_cache = False
tokenizer_llama = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer_llama.pad_token = tokenizer_llama.eos_token

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [7]:
lora_config = LoraConfig.from_pretrained('/content/drive/MyDrive/Colab_Notebooks/Хакатон_Санкт-Петербург/LLAMAcheck')
model_llama = get_peft_model(model_llama, lora_config)

In [34]:
import gradio as gr

input_component = gr.inputs.Textbox(label="Вводи")
output_component1 = gr.outputs.Label(label="GPT")
output_component2 = gr.outputs.Label(label="LLaMa2")

model_gpt = model_gpt.to(device)
model_gpt.eval()

def function_name(request):
    # Perform some operations on the input text
    with torch.no_grad(), torch.cuda.amp.autocast():
        encoded_input = tokenizer_gpt(request, return_tensors='pt', add_special_tokens=False).to(device)
        model_gpt.config.pad_token_id = model_gpt.config.eos_token_id
        output_g = model_gpt.generate(
            **encoded_input,
            num_beams=4,
            max_length=100,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    output1 = tokenizer_gpt.decode(output_g[0], skip_special_tokens=True)
    #LLaMa2
    inputs = tokenizer_llama(request, return_tensors="pt").to(device)
    outputs_l = model_llama.generate(**inputs, max_new_tokens=150)
    output2 = tokenizer_llama.decode(outputs_l[0], skip_special_tokens=True)
    return output1, output2

interface = gr.Interface(fn=function_name, inputs=input_component, outputs=[output_component1, output_component2], title="МФЦ рулезз", css="footer {visibility: hidden}")
interface.launch(share=True)

<ipython-input-34-10663d5a51b0>:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  input_component = gr.inputs.Textbox(label="Вводи")
<ipython-input-34-10663d5a51b0>:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  input_component = gr.inputs.Textbox(label="Вводи")
<ipython-input-34-10663d5a51b0>:3: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  input_component = gr.inputs.Textbox(label="Вводи")
<ipython-input-34-10663d5a51b0>:4: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output_component1 = gr.outputs.Label(label="GPT")
<ipython-input-34-10663d5a51b0>:4: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  output_component1 = gr.ou

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c4e4cdf0445c941773.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
